In [24]:
import pandas as pd
import MetaTrader5 as mt5
import pandas_ta as ta
import mplfinance as mpf
from datetime import datetime
import time
import vectorbt as vbt


In [25]:
print(mt5.__version__)

5.0.4803


In [26]:
mt5.initialize()

True

In [27]:
username = "202994639"
password = "_Gun159357258"
server = "Exness-MT5Trial7"
mt5.login(username, password, server)

account_info = mt5.account_info()

balance = account_info.balance
equity = account_info.equity

print("balance : ", balance)
print("equity : ", equity)

balance :  10000.0
equity :  10000.0


In [28]:
symbol = "BTCUSDm"

In [29]:
def get_candle(symbol, timeframe, period):
    data = mt5.copy_rates_from(symbol, timeframe, datetime.now(), period)
    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"], unit="s")

    return df

In [30]:
def bbma(df):
    df["wma5_l"] = ta.wma(df.low, length=5)
    df["wma10_l"] = ta.wma(df.low, length=10)

    df["wma5_h"] = ta.wma(df.high, length=5)
    df["wma10_h"] = ta.wma(df.high, length=10)

    df[["bb_low", "bb_mid", "bb_up"]] = df.ta.bbands(length=20).iloc[:, :-2]

    df["ema50"] = ta.ema(df.close, length=50)

    df.loc[df["bb_low"] > df["ema50"], "action"] = "buy"
    df.loc[df["bb_low"] < df["ema50"], "action"] = "sell"

    return df

In [31]:
def mtk_order(df, symbol):
    global position
    # df = df[-2:]

    if df.iloc[-1]["action"] == "buy" and position is None:
        tp = 200
        price = mt5.symbol_info_tick(symbol).ask
        tp_price = price + tp * mt5.symbol_info(symbol).point

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.01,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type": mt5.ORDER_TYPE_BUY,
            "sl": 0.0,
            "tp": tp_price,
            "deviation": 20,
            "magic": 234000,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
        print(request)

        if mt5.order_send(request):
            print("Send order buy")
        else:
            print("Not Send")

        time.sleep(1)

        # buy_log = client.get_order(symbol=coin, orderId=order['orderId'])
        # log.append(['buy',buy_log])
        position = "buy"

    elif df.iloc[-1]["action"] == "sell" and position is None:
        tp = 200
        price = mt5.symbol_info_tick(symbol).bid
        tp_price = price - tp * mt5.symbol_info(symbol).point

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.01,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type": mt5.ORDER_TYPE_SELL,
            "sl": 0.0,
            "tp": tp_price,
            "deviation": 20,
            "magic": 234000,
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_FOK,
        }
        print(request)

        if mt5.order_send(request):
            print("Send order sell")
        else:
            print("Not Send")

        time.sleep(1)

        # buy_log = client.get_order(symbol=coin, orderId=order['orderId'])
        # log.append(['buy',buy_log])
        position = "sell"

    else:
        print("No Signal")

In [32]:
def chart(df):
    # ปรับ marker_position
    df.loc[df["action"] == "buy", "marker_position"] = df["Low"] * 0.9999
    df.loc[df["action"] == "sell", "marker_position"] = df["High"] * 1.00005

    # สร้าง marker_plot และ color_plot
    marker_plot = ["^" if x == "buy" else "v" for x in df["action"]]
    color_plot = ["green" if x == "buy" else "red" for x in df["action"]]

    # สร้าง marker
    marker = mpf.make_addplot(
        df["marker_position"], type="scatter", color=color_plot, marker=marker_plot
    )

    # สร้าง add_plot
    add_plot = [
        mpf.make_addplot(df["bb_mid"], type="line", color="black"),
        mpf.make_addplot(df["bb_low"], type="line", color="black"),
        mpf.make_addplot(df["bb_up"], type="line", color="black"),
        mpf.make_addplot(df["ema50"], type="line", color="blue"),
    ]

    # รวม marker กับ add_plot
    all_plot = [marker] + add_plot

    # Plot
    mpf.plot(df, type="candle", style="yahoo", addplot=all_plot, figsize=(16, 9))

    return df

In [33]:
data = get_candle(symbol=symbol, timeframe=mt5.TIMEFRAME_M1, period=100)
data

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-02-01 03:18:00,102326.66,102362.65,102318.09,102361.47,392,2900,0
1,2025-02-01 03:19:00,102360.43,102383.16,102357.18,102376.45,358,2900,0
2,2025-02-01 03:20:00,102375.52,102403.84,102363.86,102378.80,362,2900,0
3,2025-02-01 03:21:00,102378.42,102446.17,102378.42,102429.90,359,2900,0
4,2025-02-01 03:22:00,102430.02,102434.45,102373.63,102402.52,361,2900,0
...,...,...,...,...,...,...,...,...
95,2025-02-01 04:53:00,102404.01,102410.33,102341.79,102348.31,299,2900,0
96,2025-02-01 04:54:00,102348.53,102350.65,102287.14,102302.57,320,2900,0
97,2025-02-01 04:55:00,102302.37,102315.43,102288.42,102310.70,320,2900,0
98,2025-02-01 04:56:00,102310.37,102311.70,102276.77,102278.39,286,2900,0


In [34]:
def do_all(ticker, interval=mt5.TIMEFRAME_M1, periods=100):
    data = get_candle(symbol=ticker, timeframe=interval, period=periods)

    df = data.copy()
    df = bbma(df)
    mtk_order(df, ticker)
    df = df.set_index(df["time"]).drop(columns=["time"])
    df.rename(
        columns={
            "open": "Open",
            "high": "High",
            "low": "Low",
            "close": "Close",
            "real_volume": "Volume",
        },
        inplace=True,
    )
    print(df.tail())
    if df.empty:
        print("DataFrame ว่าง ตรวจสอบแหล่งข้อมูลหรือเงื่อนไขการกรอง")
    else:
        chart(df)

    return df


In [35]:
# from IPython.display import clear_output

# log = []
# position = None
# while True:
#     # sleep to the start of next minute/second
#     clear_output(wait=True)
#     time.sleep(60 - time.time() % 60)
#     do_all(ticker=symbol, interval=mt5.TIMEFRAME_M1, periods=100)

In [36]:
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": 0.01,
    "price": mt5.symbol_info_tick(symbol).ask,
    "type": mt5.ORDER_TYPE_BUY,
    "sl": 0.0,
    "tp": 0.0,
    "deviation": 20,
    "magic": 234000,
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_FOK,
}
print(request)

if mt5.order_send(request):
    print("Send order buy")
else:
    print("Not Send")

{'action': 1, 'symbol': 'BTCUSDm', 'volume': 0.01, 'price': 102317.74, 'type': 0, 'sl': 0.0, 'tp': 0.0, 'deviation': 20, 'magic': 234000, 'type_time': 0, 'type_filling': 0}
Send order buy
